In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq
 
from bokeh.palettes import brewer, Spectral6
from bokeh.io import show, output_file, curdoc
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, Panel, Tabs, GeoJSONDataSource, LinearColorMapper
from bokeh.models import ColorBar
from bokeh.layouts import column, gridplot, row
from bokeh.transform import factor_cmap
from bokeh.models import NumeralTickFormatter, Title, Label, Paragraph, Div, CustomJSHover, BoxAnnotation

In [9]:
df_steel = pd.read_parquet('./data/canada-steel-US-exports-031225.parquet')

df_fentl = pd.read_parquet('./data/canada-US-exports-030425.parquet')


In [21]:
df_steel.sort_values(by = ["exports"], ascending = False)[0:20]

,exports,description,hs-code,tariff,export_share,color
711291,3.852355e+09,WASTE & SCRAP GOLD EXCL SWPNGS CNTNG OTH PREC ...,711291,25.0,96.988731,#ff0000
851762,2.789308e+09,MACH FOR RECP/CONVR/TRANS/REGN OF VOICE/IMAGE/...,851762,25.0,12.377326,#ff0000
847130,1.924976e+09,PORT DIGTL AUTOMATIC DATA PROCESS MACH NOT > 1...,847130,25.0,28.947967,#ff0000
851713,1.400582e+09,SMARTPHONES,851713,25.0,14.564110,#ff0000
760612,1.003747e+09,"ALUMINUM ALLOY RECT PLATES ETC, OVER .2 MM THICK",760612,25.0,35.576448,#ff0000
732690,8.156968e+08,ARTICLES OF IRON OR STEEL NESOI,732690,25.0,13.625715,#ff0000
950300,7.647833e+08,"TRICYCLE, SCOOTR, PEDAL CAR & SIM WHEELED TYS;...",950300,25.0,62.103975,#ff0000
940199,6.798869e+08,"PARTS OF SEATS, NESOI",940199,25.0,31.756389,#ff0000
731815,6.204336e+08,THREADED SCREWS AND BOLTS NESOI OF IRON OR STEEL,731815,25.0,24.821158,#ff0000
710812,6.197587e+08,"GOLD, NONMONETARY, UNWROUGHT NESOI",710812,25.0,2.134695,#ff0000


In [14]:
def make_source(df):
    
    df["position"] = df.reset_index().index.values
        
    df["hover_label"] = (df["exports"]/1000000000).map('{:,.1f}'.format)
    
    df["hover_label_2"] = (df["export_share"]).map('{:,.1f}'.format)
    
    df["hover_label_3"] = (df["tariff"]).map('{:,.1f}'.format)
    
    source = ColumnDataSource(df)
    
    return source

In [12]:
def make_bar_chart(df):

    width = 600
    height = 500

    source = make_source(df)
        
    p = figure(plot_height=height, plot_width = width, title= "Top US Exports to Canada Subject to Retaliatory Tariffs",
           toolbar_location = 'below',
           tools = "reset")
        
    p.vbar(x = "position", top = "exports", width = 0.6, alpha = 0.65,
       hatch_pattern = " ",hatch_alpha = 0.10, color = "color",
       source = source)
    
    y_custom = CustomJSHover(code=""" return '' + special_vars.data_y
            """)

##########################################################################
    TIMETOOLTIPS = """
    <div style="background-color:#F5F5F5; opacity: 0.95; border: 0px 0px 0px 0px">
        <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold">@description</span>
        </div>
        <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold">2024 Exports: $@hover_label Billion</span>
        </div>
        <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold">Share of Total: @hover_label_2%</span>
        </div>
        <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold">Applied Tariff: @hover_label_3%</span>
        </div>
    </div>
    """

    p.add_tools(HoverTool(tooltips = TIMETOOLTIPS))
##########################################################################

    #p.ygrid.grid_line_color = None
    p.xgrid.grid_line_color = None
    
    p.title.text_font_size = '13pt'
    p.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
    p.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks
    p.xaxis.major_label_text_font_size = '0pt'  # turn off x-axis tick labels

    p.yaxis.formatter = NumeralTickFormatter(format="($0.0 a)")
    p.yaxis.minor_tick_line_color = None
    p.y_range.start = 0 
    
    p.y_range.end = df.exports.max() + 0.10*df.exports.max()
    
    p.border_fill_color = background    
    
    p.background_fill_color = background 
    p.background_fill_alpha = 0.75    
    
    p.toolbar.autohide = True
    
    p.outline_line_color = None
    p.sizing_mode= "scale_both"
    p.max_height = height
    p.max_width = width
    p.min_height = int(0.25*height)
    p.min_width = int(0.25*width)
    return p

In [16]:
crl = ["darkblue","slategray","slategray","crimson","crimson"]

background = "#ffffff"

In [22]:
p1 = make_bar_chart(df_steel.sort_values(by = ["exports"], ascending = False)[0:20])

p2 = make_bar_chart(df_fentl.sort_values(by = ["exports"], ascending = False)[0:20])

In [23]:
tab1 = Panel(child= p1, title="Steel Retaliation")

tab2 = Panel(child= p2, title="Fentanyl Retaliation")

output_file('.\\docs\\' + "canada-retaliation-steel.html")

div0 = Div(text = """Each bar represents the total sum of U.S. exports in 2024 for a HS6 category subject to Canadaian tariffs. 
Only max 20 categories are displayed. Hover your cursor over each bar to learn more.
""", max_width=600, background = background )

div0.sizing_mode= "scale_both"
        
outfig = column(Tabs(tabs=[tab1, tab2], tabs_location = "above"), div0, sizing_mode="scale_both")

show(outfig)